In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook

In [3]:
U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [4]:
TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
class CharDataset(Dataset):
    def __init__(self, chats, max_len=32):
        self._data = chats
        self.first = True
        self.q_token = U_TKN
        self.a_token = S_TKN
        self.sent_token = SENT
        self.bos = BOS
        self.eos = EOS
        self.mask = MASK
        self.pad = PAD
        self.max_len = max_len
        self.tokenizer = TOKENIZER 

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        turn = self._data.iloc[idx]
        q = turn['Q']
        a = turn['A']
        #sentiment = str(turn['label'])
        q_toked = self.tokenizer.tokenize(self.q_token + q)   
        q_len = len(q_toked)
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len
            if a_len <= 0:
                q_toked = q_toked[-(int(self.max_len/2)):]
                q_len = len(q_toked)
                a_len = self.max_len - q_len
                assert a_len > 0
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)
            assert a_len == len(a_toked), f'{a_len} ==? {len(a_toked)}'
        # [mask, mask, ...., mask, ..., <bos>,..A.. <eos>, <pad>....]
        labels = [
            self.mask,
        ] * q_len + a_toked[1:]
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        self.max_len
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]
        return(token_ids, np.array(mask),
               labels_ids)

In [6]:
class KoGPT2Chat(nn.Module):
    def __init__(self):
        super(KoGPT2Chat, self).__init__()
        self.neg = -1e18
        self.kogpt2 = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
        self.loss_function = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs):
        # (batch, seq_len, hiddens)
        output = self.kogpt2(inputs, return_dict=True)
        return output.logits

In [7]:
model = KoGPT2Chat().cuda()

In [8]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [9]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [10]:
loss_fn = nn.CrossEntropyLoss()

In [11]:
data = pd.read_csv('/content/drive/MyDrive/자연어처리음성인식/Trash/concatData.csv')

In [12]:
train_set = CharDataset(data, max_len=32)

In [13]:
def _collate_fn(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [14]:
train_dataloader = DataLoader(train_set, batch_size=96, num_workers=2,shuffle=True, collate_fn=_collate_fn)

In [15]:
num_epochs = 20

In [16]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * 2)

In [17]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [18]:
neg = -1e18

In [19]:
log_interval = 100

In [ ]:
max_loss = 10000
for e in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_id, (token_ids, mask, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().cuda()
        mask = mask.long().cuda()
        label = label.long().cuda()
        out = model(token_ids)
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, neg * torch.ones_like(out))
        loss = loss_fn(mask_out.transpose(2, 1), label)
        loss.backward()
        train_loss += loss
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        if batch_id % log_interval == 0:
            print("EPOCH {} [{}/{}]  >>>  loss : {:.6f}\t  train_loss : {:.3f}".format(e+1, batch_id+1,len(train_dataloader),
                                                                           loss.data.cpu().numpy(), train_loss / (batch_id+1)))
    print("EPOCH {}  >>>  loss : {:.6f}\t  train_loss : {:.3f}".format(e+1, loss.data.cpu().numpy(), train_loss / (batch_id+1)))
    if max_loss > train_loss / (batch_id+1):
      max_loss = train_loss / (batch_id+1)
      torch.save(model.state_dict(), '/content/drive/MyDrive/자연어처리음성인식/조상연/TorchGPT2.pt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1324 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """


EPOCH 1 [1/1324]  >>>  loss : 9.004754	  train_loss : 9.005
EPOCH 1 [101/1324]  >>>  loss : 8.962007	  train_loss : 8.945
EPOCH 1 [201/1324]  >>>  loss : 8.594399	  train_loss : 8.842
EPOCH 1 [301/1324]  >>>  loss : 8.380759	  train_loss : 8.720
EPOCH 1 [401/1324]  >>>  loss : 8.354620	  train_loss : 8.591
EPOCH 1 [501/1324]  >>>  loss : 8.223542	  train_loss : 8.470
EPOCH 1 [601/1324]  >>>  loss : 8.001460	  train_loss : 8.372
EPOCH 1 [701/1324]  >>>  loss : 7.972234	  train_loss : 8.291
EPOCH 1 [801/1324]  >>>  loss : 7.834918	  train_loss : 8.225
EPOCH 1 [901/1324]  >>>  loss : 7.780523	  train_loss : 8.171
EPOCH 1 [1001/1324]  >>>  loss : 7.589610	  train_loss : 8.121
EPOCH 1 [1101/1324]  >>>  loss : 7.794427	  train_loss : 8.080
EPOCH 1 [1201/1324]  >>>  loss : 7.839632	  train_loss : 8.043
EPOCH 1 [1301/1324]  >>>  loss : 7.774604	  train_loss : 8.011
EPOCH 1  >>>  loss : 7.561442	  train_loss : 8.004


  0%|          | 0/1324 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """


EPOCH 19 [1/1324]  >>>  loss : 7.158290	  train_loss : 7.158
EPOCH 19 [101/1324]  >>>  loss : 7.139378	  train_loss : 7.059
EPOCH 19 [201/1324]  >>>  loss : 7.166185	  train_loss : 7.064
EPOCH 19 [301/1324]  >>>  loss : 7.155019	  train_loss : 7.063
EPOCH 19 [401/1324]  >>>  loss : 7.067724	  train_loss : 7.062
EPOCH 19 [501/1324]  >>>  loss : 7.050869	  train_loss : 7.063
EPOCH 19 [601/1324]  >>>  loss : 7.097532	  train_loss : 7.069
EPOCH 19 [701/1324]  >>>  loss : 7.351290	  train_loss : 7.068
EPOCH 19 [801/1324]  >>>  loss : 7.043020	  train_loss : 7.069
EPOCH 19 [901/1324]  >>>  loss : 7.118125	  train_loss : 7.072
EPOCH 19 [1001/1324]  >>>  loss : 7.005401	  train_loss : 7.072
EPOCH 19 [1101/1324]  >>>  loss : 7.062823	  train_loss : 7.073
EPOCH 19 [1201/1324]  >>>  loss : 7.072440	  train_loss : 7.075
EPOCH 19 [1301/1324]  >>>  loss : 7.271628	  train_loss : 7.078
EPOCH 19  >>>  loss : 7.256765	  train_loss : 7.079


  0%|          | 0/1324 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """


EPOCH 20 [1/1324]  >>>  loss : 7.154314	  train_loss : 7.154
EPOCH 20 [101/1324]  >>>  loss : 7.040008	  train_loss : 7.024
EPOCH 20 [201/1324]  >>>  loss : 7.018530	  train_loss : 7.027
EPOCH 20 [301/1324]  >>>  loss : 7.040117	  train_loss : 7.034
EPOCH 20 [401/1324]  >>>  loss : 7.071582	  train_loss : 7.037
EPOCH 20 [501/1324]  >>>  loss : 7.141993	  train_loss : 7.039
EPOCH 20 [601/1324]  >>>  loss : 6.939913	  train_loss : 7.040
EPOCH 20 [701/1324]  >>>  loss : 7.094103	  train_loss : 7.043
EPOCH 20 [801/1324]  >>>  loss : 7.122448	  train_loss : 7.048
EPOCH 20 [901/1324]  >>>  loss : 7.019066	  train_loss : 7.048
EPOCH 20 [1001/1324]  >>>  loss : 7.075564	  train_loss : 7.048
EPOCH 20 [1101/1324]  >>>  loss : 7.305051	  train_loss : 7.050
EPOCH 20 [1201/1324]  >>>  loss : 7.143764	  train_loss : 7.052
EPOCH 20 [1301/1324]  >>>  loss : 7.251099	  train_loss : 7.055
EPOCH 20  >>>  loss : 6.955148	  train_loss : 7.056
